# Assignment: Extracting Static WebPage

Extract information about “วันพระ” for 3 years from:
- https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2565.aspx
- https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2566.aspx
- https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2567.aspx


Note that you can use dateparse package to parse Thai date.  First, we will have to install the package, this is for Google Colab users.  Otherwise, installing via command line is recommended (pip or conda).

In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    %pip install dateparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 KB 5.2 MB/s eta 0:00:00


In [2]:
import dateparser

To convert from thai text date string, we will use the parse method.  Note that the parse method assumes the BC year, not BE.  Thus, we will have to subtract 543 from the year.  In addition, weekday() returns day of week with 0=Monday, ..., 6=Sunday.

In [3]:
dt = dateparser.parse('วันศุกร์ที่ 17 มกราคม 2563')

# this will print out weekday == 0 (Monday)
print(dt)
print(dt.weekday())

# this will print out weekday == 4 (Friday)
dt = dt.replace(year=dt.year-543)
print(dt)
print(dt.weekday())

2563-01-17 00:00:00
0
2020-01-17 00:00:00
4


In [4]:
dt = dateparser.parse('วันเสาร์ที่ 21 กันยายน 2564')
dt = dt.replace(year=dt.year-543)
print(dt)
print(dt.weekday())

2021-09-21 00:00:00
1


Count the distribution of number of week days that are “วันพระ” for all three years and answer the following questions:

## How many วันพระ in total (of 3 years)?

In [19]:
import requests,re
from bs4 import BeautifulSoup
bigdata = requests.get('https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2565.aspx')
bigdata2 = requests.get('https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2566.aspx')
bigdata3 = requests.get('https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2567.aspx')
len(bigdata.text)

56473

In [20]:
soup = BeautifulSoup(bigdata.text,"lxml")
soup2 = BeautifulSoup(bigdata2.text,"lxml")
soup3 = BeautifulSoup(bigdata3.text,"lxml")
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   วันพระ 2565 ปฏิทินวันพระ พ.ศ.2565/2022 - myhora.com
  </title>
  <meta content="วันพระ 2565 ปฏิทินวันพระ 2565/ 2022 ปฏิทินวันพระตลอดปี 2565 วันธรรมสวนะหรือวันอุโบสถ ได้แก่ วันขึ้น ๘ ค่ำ, วันวันเพ็ญขึ้น ๑๕ ค่ำ , วันแรม ๘ ค่ำ และวันแรม ๑๕ ค่ำ หากเดือนใดเป็นเดือนขาด ถือเอาวันแรม ๑๔ ค่ำ วันมาฆบูชา วิสาขบูชา วันอาสาฬหบูชา วันเข้าพรรษา วันออกพรรษา" name="Description"/>
  <meta content="ปฏิทินวันพระ 2565,วันพระ 2022,วันพระ พ.ศ.2565,Calendar 2022,ปฏิทินปี 2565,ปฏิทินจันทรคติไทย พ.ศ.2565,ปฏิทินวันพระ 2565,ปฏิทินวันหยุดราชการ 2565 ,วันหยุดชดเชย,วันพระ 8 ค่ำ,วันพระ 15 ค่ำ,วันขึ้นปีใหม่,วันมาฆบูชา 2565,วิสาขบูชา 2565,วันอาสาฬหบูชา 2565,วันเข้าพรรษา 2565,วันออกพรรษา 2565,วันลอยกระทง 2565, Calendar 2022" name="Keywords"/>
  <meta content="10 Days" name="Revisit-After"/>
  <meta content="index,follow" name="robots"/>
  <meta content="noodp" name="robots"/>
  <meta content="MYHORA.com, Thailand" name="author"/>
  <meta content="787285358050657" property="fb:

In [21]:
len(soup.find_all(attrs={'class': 'bud-day'}))+len(soup2.find_all(attrs={'class': 'bud-day'}))+len(soup3.find_all(attrs={'class': 'bud-day'}))

152

## How many days in total (of 3 years) that วันพระ is Monday?

In [27]:
num=0
for i in soup.find_all(attrs={'class': 'bud-day'}):
  if "จันทร์" in str(i):
    num+=1
for i in soup2.find_all(attrs={'class': 'bud-day'}):
  if "จันทร์" in str(i):
    num+=1
for i in soup3.find_all(attrs={'class': 'bud-day'}):
  if "จันทร์" in str(i):
    num+=1
print(num)

21


## Which day of the week that has the minimum number of วันพระ?

In [40]:
import numpy as np
def phraday(soup):
  num=np.zeros(7,dtype=int)
  day=list(["อาทิตย์","จันทร์","อังคาร","พุธ","พฤหัส","ศุกร์","เสาร์"])
  for i in soup:
    for j in range(7):
      if day[j] in str(i):
        num[j]+=1
        
  return num
phradays=phraday(soup.find_all(attrs={'class': 'bud-day'}))+phraday(soup2.find_all(attrs={'class': 'bud-day'}))+phraday(soup3.find_all(attrs={'class': 'bud-day'}))
print(phradays)

[24 21 20 22 23 21 21]


In [42]:
day=list(["อาทิตย์","จันทร์","อังคาร","พุธ","พฤหัส","ศุกร์","เสาร์"])
min=1000
for i in range(7):
  if(min>phradays[i]):
    min=phradays[i]
    minday=day[i]
print(minday)

อังคาร


## Which day of the week that has the maximum number of วันพระ?

In [45]:
max=0
for i in range(7):
  if(max<phradays[i]):
    max=phradays[i]
    maxday=day[i]
print(maxday)

อาทิตย์
